# Taking vacations on the best possible weather conditions

## Dependecies and configuration

In [3]:
# Import libraries
import pandas as pd
import requests
import os
import gmaps

# Import keys.
from api_keys import g_key

# Weather information for 1000 cities dataset file
one_thousand_cities_weather_data_path = os.path.join("output", "one_thousand_unique_cities.csv")

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Base query for Goodle nearbysearch API.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

## Load datasets and prepare parameters

In [4]:
# Read weather information for 1000 cities dataset.
cities_df = pd.read_csv(one_thousand_cities_weather_data_path)

# Create lists' parameters for google maps
locations = cities_df[["lat", "lon"]].astype(float)
weather_humidity = cities_df["humidity"].astype(float)


## Create map to show humidity in a 1000 cities

In [5]:
# Create a heat map showing the humidity for a list of cities

humidity_fig = gmaps.figure()

humidity_heat_layer = gmaps.heatmap_layer(
    locations,
    weights = weather_humidity,
    dissipating = False,
    max_intensity = 100,
    point_radius = 1
)

humidity_fig.add_layer(humidity_heat_layer)

humidity_fig


Figure(layout=FigureLayout(height='420px'))

## Look up the cities with the best weather

In [6]:
# Look up cities in the dataframe according to certain conditions

best_weather_conditions_cities = cities_df[
    (cities_df['temperature'] > 65) & (cities_df['temperature'] < 75)
    & (cities_df['wind_speed'] < 10)
    & (cities_df['cloudiness'] == 0)
]

best_weather_conditions_cities = best_weather_conditions_cities.drop(columns = ['index', 'level_0'])
best_weather_conditions_cities = best_weather_conditions_cities.reset_index(drop = True)
best_weather_conditions_cities

,id,name,state,country,lon,lat,temperature,humidity,cloudiness,wind_speed,weather_condition,weather_condition_desc
0,5373327,Milpitas,CA,US,-121.906616,37.428268,66.70,35.0,0.0,1.01,Clear,clear sky
1,4898875,La Salle,IL,US,-89.091751,41.333370,67.03,63.0,0.0,1.01,Clear,clear sky
2,5401121,Tara Hills,CA,US,-122.316360,37.993530,65.89,71.0,0.0,5.32,Clear,clear sky
3,6537556,Polignano a Mare,NaN,IT,17.215891,40.995930,72.73,63.0,0.0,1.99,Clear,clear sky
4,256485,Néon Monastírion,NaN,GR,22.250000,39.233330,70.18,44.0,0.0,6.93,Clear,clear sky
5,6323793,Alta Floresta,NaN,BR,-56.302879,-10.019970,68.58,74.0,0.0,4.72,Clear,clear sky
6,3469294,Bonito,NaN,BR,-56.481941,-21.121111,67.35,70.0,0.0,7.09,Clear,clear sky
7,3693305,Pueblo Nuevo,NaN,PE,-79.883331,-5.716670,65.19,78.0,0.0,2.24,Clear,clear sky
8,3401443,Cruzeta,NaN,BR,-36.789719,-6.411670,67.37,83.0,0.0,6.67,Clear,clear sky
9,3392887,Patos,NaN,BR,-37.279999,-7.024440,65.08,83.0,0.0,5.55,Clear,clear sky


## Look up hotels near the cities with the best weather

In [7]:
# Interact with Google API to get data on one hotel close to each city in the list of cities with ideal weather.

# Fix parameter values to call the API
target_search = "hotel"
target_type = "lodging"
target_radius = 5000

# Parameters for the query to google nearbysearch API.
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

counter = 0

# Call the google API for each city with ideal weather conditions.
print("Starting data request...")
for index, city in best_weather_conditions_cities.iterrows() :
    counter += 1
    
    target_coordinates = f"{city['lat']},{city['lon']}"
    params.update({"location": target_coordinates})
    
    print(f"Requesting data for: {counter} - {city['name']}, {city['country']} (lat: {city['lat']}, lon: {city['lon']})...")
    response = requests.get(base_url, params=params)
    hotels_data = response.json()
    
    # Only get data from the cities that had a hotel within 5000
    if hotels_data['status'] == "OK" :
        best_weather_conditions_cities.loc[index, 'hotel_name'] = hotels_data['results'][0]['name']
        best_weather_conditions_cities.loc[index, 'hotel_vicinity'] = hotels_data['results'][0]['vicinity']
        print("Data received.")
    else :
        print("Data not available.")
print("Data request completed.")

# Remove all cities that didn't have a hotel within 5000 meters.
best_weather_conditions_cities.dropna(subset = ['hotel_name'], inplace = True)

Starting data request...
Requesting data for: 1 - Milpitas, US (lat: 37.428268, lon: -121.90661599999999)...
Data received.
Requesting data for: 2 - La Salle, US (lat: 41.33337, lon: -89.091751)...
Data received.
Requesting data for: 3 - Tara Hills, US (lat: 37.99353, lon: -122.31636)...
Data received.
Requesting data for: 4 - Polignano a Mare, IT (lat: 40.99593, lon: 17.215891)...
Data received.
Requesting data for: 5 - Néon Monastírion, GR (lat: 39.23333, lon: 22.25)...
Data not available.
Requesting data for: 6 - Alta Floresta, BR (lat: -10.019969999999999, lon: -56.302879000000004)...
Data not available.
Requesting data for: 7 - Bonito, BR (lat: -21.121111, lon: -56.481941000000006)...
Data received.
Requesting data for: 8 - Pueblo Nuevo, PE (lat: -5.71667, lon: -79.88333100000001)...
Data not available.
Requesting data for: 9 - Cruzeta, BR (lat: -6.41167, lon: -36.789719)...
Data not available.
Requesting data for: 10 - Patos, BR (lat: -7.024439999999999, lon: -37.279999)...
Data 

In [8]:
locations = best_weather_conditions_cities[["lat", "lon"]].astype(float)
weather_humidity = best_weather_conditions_cities["humidity"].astype(float)
maps_info_box = [
    f"<B>Hotel name</B> <BR> {city['hotel_name']} <BR> \
    <B>City</B> <BR> {city['name']} <BR> \
    <B>Country</B> <BR> {city['country']} <BR> \
    <B>Weather condition</B> <BR> {city['weather_condition_desc']}" \
    for index, city in best_weather_conditions_cities.iterrows()
]

hotels_fig = gmaps.figure()

hotels_heat_layer = gmaps.heatmap_layer(
    locations,
    weights = weather_humidity,
    dissipating = False,
    max_intensity = 100,
    point_radius = 1
)

hotels_markers = gmaps.marker_layer(
        locations,
        info_box_content = maps_info_box
)

hotels_fig.add_layer(hotels_heat_layer)
hotels_fig.add_layer(hotels_markers)

hotels_fig

Figure(layout=FigureLayout(height='420px'))